### HTML 다루기
- Top-down 방식 : 큰 그림부터 이해하고 세부적으로, 나무를 보지 말고 숲을 보자
- Bottom-UP 방식 : 세부적인 데서 출발하자

In [ ]:
import requests
import bs4

In [ ]:
url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1'
response = requests.get(url).text

one_page = bs4.BeautifulSoup(response)

In [ ]:
title_list = one_page.find_all('b')

In [ ]:
title_list

In [ ]:
book_list=[]
for title in title_list[0::3]:
    book_list.append(title.text)

In [ ]:
book_list

In [ ]:
top_down = one_page.find_all('div', {'class':'ss_book_box'})
one_book = top_down[0].find('div',{'class' : 'ss_book_list'})
one_book.find('a', 'bo3').find('b').text

# top-down 방식, 위에서 부터 차례로 내려오며 찾는 방식, 더 확실하게 찾을 수 있다.

In [ ]:
# 함수만들기
def best_books():
    """
    알라딘 사이트에서 베스트 셀러 50권의 이름을 반환합니다.
    """
    url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1'
    response = requests.get(url).text
    one_page = bs4.BeautifulSoup(response)
    title_list = one_page.find_all('b')
    
    best_selling_book_list = []
    for title in title_list[0::3]:
        best_selling_book_list.append(title.text)

    return best_selling_book_list

In [ ]:
test = best_books()
#return : 알라딘 사이트의 베스트 셀러 50권의 책 제목이 반환

In [ ]:
test

### select() 를 이용한 검색


- 태그를 찾을 때, 몇몇 사이트는 검색을 못하게 막을 수도 있다.
- select()는 find_all()과 유사한 기능을 제공함
- 보다 유연하게 활용될 수 있다, 더 쉽다 카더라
- CSS 선택자의 형식에 따라 태그를 검색함

In [ ]:
one_page.select(".bo3")

In [ ]:
one_page.select('#newbg_body')

In [ ]:
one_page.select('div.ss_book_list a.bo3 b')

In [ ]:
one_page.select('link + style') # link 태그 다음에 style 태그

In [ ]:
one_page.select('div > ul') # div 바로 밑에 있는 ul 태그

In [ ]:
one_page.select('div.ss_book_list > ul > li')

In [ ]:
one_page.select('a[href^=https]') # href 속성값이 https로 시작하는 a태그를 찾아줘

In [ ]:
# select를 활용하여 책 제목을 받아보자
book_name_list = []
for name in one_page.select('a.bo3 b'):
    book_name_list.append(name.text)

book_name_list

#### 1. 네이버 웹툰 데이터 크롤링 : 네이버, 웹툰 제목, 번호 추출 -> 웹툰의 리스트를 출력, 다운로드 받을 웹툰을 입력-> 다운로드 받을 몇화 입력, 웹툰 다운로드
#### 2. 초성퀴즈 : 랜덤으로 주는 초성, 입력받은 단어가 다음 사전에 결과가 나오는지 크롤링하여 정답 체크 다음 사전 초성은 두글자
##### 3. 컨탠츠 댓글 수집 : 사용자가 특정 키워드에 대한 컨탠츠를 찾고 해당 컨탠츠의 댓글 수집

In [ ]:
!pip install jamo

In [1]:
from jamo import h2j, j2hcj  # 글자 분해하는 라이브러리

import random as r

# 초성모음
CHO = [
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 
    'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]

#예시 코드
allInit = []
text = "삼성전자"
for x in text:
    temp = h2j(x)
    imf = j2hcj(temp)  # init,middle,final
    print(f"{temp}, {imf}")
    allInit.append(imf[0])
            
print("".join(allInit))


r.randint(0, 18)

print(imf[0] in CHO)
print(len(CHO))

삼, ㅅㅏㅁ
성, ㅅㅓㅇ
전, ㅈㅓㄴ
자, ㅈㅏ
ㅅㅅㅈㅈ
True
19


In [3]:
def give_chosung(num=2):
    """
        랜덤한 초성을 리스트로 반환(기본 사이즈 2)
    """
    chosung = []
    for i in range(0, num):
        chosung.append(CHO[r.randint(0,18)])
        # print(chosung[i])
    return chosung 

In [310]:
question = give_chosung()

In [312]:
question

['ㅁ', 'ㅈ']

In [276]:
import requests
import bs4

def input_word():
    return input("단어를 입력하세요(종료:q)")

# 입력한 단어 길이 체크
def is_same_len(word, length):
    return len(word) == length

# 입력한 단어 초성 체크
def is_right_chosung(word, *chosung):
    
    for i in range(0, len(word)):
        if j2hcj(h2j(word[i]))[0] != chosung[i]:
            return False

    return True

def is_right_word(word):
    url= f"https://dic.daum.net/search.do?q={word}"
    response = requests.get(url).text
    search_result = bs4.BeautifulSoup(response)

    try:
        first_word = search_result.find('div','search_box').find('a', 'txt_cleansch').find('span', 'txt_emph1').text
        if first_word == word:
            return True
        else:
            print('사전에 없는 단어입니다')
            return False
    except :
        print('사전에 없는 단어입니다')
        return False
    


In [304]:

print(is_same_len(word, len(question)))
print(is_right_chosung(word, *question))
question

True
True


['ㄸ', 'ㄲ']

In [218]:
test = is_right_word("지구")

In [220]:
test

True

In [408]:
out_word=[]
score = 0
question = give_chosung()
print("초성은", question)


while True:
    
    word = input_word()

    if word == "q":
        break
    elif word in out_word:
        print("이미 사용한 단어입니다")
    elif not is_same_len(word, len(question)):
        print("길이가 다릅니다")
    elif not is_right_chosung2(word, *question):
        print("초성이 다릅니다")
    elif is_right_word(word):
        print("정답입니다!")
        out_word.append(word)
        score+=1
    else:
        pass


print("게임을 종료합니다. 점수:", score)

초성은 ['ㄴ', 'ㄲ']


단어를 입력하세요(종료:q) 누꼬


사전에 없는 단어입니다


단어를 입력하세요(종료:q) 니꺼


사전에 없는 단어입니다


단어를 입력하세요(종료:q) 뇨끼


정답입니다!


단어를 입력하세요(종료:q) q


게임을 종료합니다. 점수: 1


In [406]:
# 유니코드 초성 검증기
consonants = {'ㄱ': 44032, 'ㄲ': 44620, 'ㄴ': 45208, 'ㄷ': 45796, 'ㄸ': 46384, 'ㄹ': 46972, 'ㅁ': 47560, 
              'ㅂ': 48148, 'ㅃ': 48736, 'ㅅ': 49324, 'ㅆ': 49912, 'ㅇ': 50500, 'ㅈ': 51088, 'ㅉ': 51676, 'ㅊ': 52264, 
              'ㅋ': 52852, 'ㅌ': 53440, 'ㅍ': 54028, 'ㅎ': 54616}

def is_right_chosung2(word, *chosung):
    for i in range(0, len(word)):
        cho = [k for k, v in consonants.items() if v == (ord(word[i]) - ((ord(word[i]) - 520) % 588))]
        if cho[0] != chosung[i]:
            return False

    return True


#     user_word_1 = ord(word[0]) - ((ord(word[0]) - 520) % 588)
#     user_word_2 = ord(word[1]) - ((ord(word[1]) - 520) % 588)

#     cho1 = [k for k, v in consonants.items() if v == user_word_1]
#     cho2 = [k for k, v in consonants.items() if v == user_word_2]
#     print(cho1, cho2)

# is_right_chosung2(input())


In [404]:

word= input()
user_word_1 = ord(word[0]) - ((ord(word[0]) - 520) % 588)
user_word_2 = ord(word[1]) - ((ord(word[1]) - 520) % 588)
print(user_word_1, user_word_2)

 ㅇ ㅋ


12280 -68


In [402]:
print(question)
is_right_chosung2("바카", *question)

['ㅂ', 'ㅋ']


True

In [360]:
print(ord('싸') - ((ord('싸') - 520) % 588))

49912
